In [3]:
!pip install Konlpy > /dev/null

In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [6]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


1. 데이터 전처리
- Train

In [7]:
# Null 데이터가 있는지 확인
train_df.document.isna().sum()

5

In [8]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [9]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [10]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [11]:
# 긍정/부정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- Test

In [12]:
# Null 데이터 확인
test_df.document.isna().sum()

3

In [13]:
# Null 데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.shape

(49997, 3)

In [14]:
# 중복 확인
test_df.document.nunique()

49157

In [15]:
# 중복 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [16]:
# Label 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

2. 텍스트 전처리
- Train

In [17]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [18]:
# '' 만 남은 데이터 --> np.nan 으로 변경시킨 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [19]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- Test

In [20]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

- 전처리가 끝난 데이터는 저장

In [22]:
train_df.to_csv('/naver_movie_train_전처리완료.csv', sep='\t', index=False)
test_df.to_csv('/naver_movie_test_전처리완료.csv', sep='\t', index=False)

3. 한글처리

In [24]:
from konlpy.tag import Okt 
okt = Okt()

In [25]:
text = '흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [26]:
# 형태소로 tokenize
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [32]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()

In [33]:
from tqdm.notebook import tqdm
X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_train.append(tmp_str)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [34]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [36]:
y_train = train_df.label.values 
y_test = test_df.label.values 

4. Feature 변환 + 모델 학습/평가

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [41]:
cvect = CountVectorizer()
lr=LogisticRegression(random_state=2022)
pipeline = Pipeline([('CVECT', cvect), ('LR', lr)])
%time pipeline.fit(X_train, y_train)

CPU times: user 9.33 s, sys: 5.28 s, total: 14.6 s
Wall time: 13.8 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [42]:
pipeline.score(X_test, y_test)

0.8273970359453042

5. 실제 데이터 테스트

In [43]:
review1= '모든 국민이 봤으면 하는 영화입니다.'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'

In [44]:
# 전처리
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review1).strip()
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review2).strip()

In [45]:
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if word not in stop_text])
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if word not in stop_text])

In [46]:
pipeline.predict([review1, review2])

array([1, 0])

6. 최적 parameter 찾기

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
params = {
    'CVECT__ngram_range': [(1,1), (1,2)],
    'CVECT__max_df': [0.95, 0.98],
    'LR__C': [1, 5]
}

In [49]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1
)
%time grid_pipe.fit(X_train, y_train)

CPU times: user 50.2 s, sys: 15.2 s, total: 1min 5s
Wall time: 3min 15s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5]},
             scoring='accuracy')

In [50]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1}

In [51]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8477032670105625